## 월간 보고용 : 주문수 분석

In [1]:
import pandas as pd
import re
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import apriori, association_rules
import datetime as dt
from dateutil.relativedelta import relativedelta
import os
from collections import Counter  

### 0. 기본설정 : 저장 위치 및 함수

In [2]:
input_path = 'C:/GMG_R&D/3. 장바구니 분석/1. 한총' 
os.chdir(input_path)
path=os.getcwd()
print(path)

x = dt.datetime.now()
file_name=f'월간보고서_{x.year}년_{(x - relativedelta(months=1)).month}월/월간_{x.month}월 {x.day}일'

if not os.path.exists(file_name):
    os.makedirs(file_name) 

C:\GMG_R&D\3. 장바구니 분석\1. 한총


In [3]:
#########  필요한 함수 정의 #########
#1. 데이터셋 만들기
def dataset_make(dic_category, month_df):
    dataset = []   
    for i in range(len(month_df)) :
        sku = month_df['Model Code (v41)'][i]
        revised = sku.split(",")
        code = []
        for j in range(len(revised)) :
            code.append(dic_category[revised[j]])   
        for k in range(month_df[month_df.columns[1]][i]) :  
            dataset.append(code) 
    return dataset

#2. 주문건수
def order_1depth(new_rex, dataset):
    per_order={}
    for PRODU in new_rex['Category1D']:
        df_count=[s for s in dataset if any(PRODU in l for l in s)] 
        per_order[PRODU]=len(df_count)

    daf=pd.DataFrame.from_dict(per_order, orient="index").reset_index()
    daf_1=daf.rename({'index':'Category1D',0:"count"},axis=1)
    return daf_1

#3. 제품건수
def product_2depth(new_rex, dataset):
    per_product={}

    for PRODU in new_rex['Category2D']:
        df_count=[s for s in dataset for i in s if PRODU in i]
        per_product[PRODU]=len(df_count)

    daf_product=pd.DataFrame.from_dict(per_product, orient="index").reset_index()
    final=pd.merge(new_rex[["Category1D","Category2D"]], daf_product, left_on="Category2D", right_on='index')
    final=final.drop(columns=["index"])
    final=final.rename({0:"count"},axis=1)
    mon_final=final.sort_values(["Category1D","count"],ascending=False)
    return mon_final


def frozen_convert(sets) : 
    return [list(sets)]


#4. 장바구니 분석
def my_basket(dataset):
    te = TransactionEncoder()
    te_ary = te.fit(dataset).transform(dataset) #True / False
    df2 = pd.DataFrame(te_ary, columns=te.columns_)
    itemsets_data = apriori(df2, min_support=0.00001, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
    rules = association_rules(itemsets_data, metric='lift', min_threshold=1).sort_values('lift',ascending=False).reset_index(drop=True)
    rules['count']=(rules['support']*len(dataset)).astype(int)
    basket = rules[rules.apply(lambda x:True if len(x.antecedents) == int(1) else False, axis=1) & rules.apply(lambda x:True if len(x.consequents) == int(1) else False, axis=1)]
    basket = basket.sort_values('confidence', ascending=False)
    basket[['antecedents','consequents']] =basket[['antecedents','consequents']].applymap(lambda x: frozen_convert(x))
    basket['antecedents']=basket['antecedents'].astype(str).apply(lambda x: x.replace('[[','['))
    basket['antecedents']=basket['antecedents'].apply(lambda x: x.replace(']]',']'))
    basket['consequents']=basket['consequents'].astype(str).apply(lambda x: x.replace('[[','['))
    basket['consequents']=basket['consequents'].apply(lambda x: x.replace(']]',']'))
    basket=basket[['antecedents','consequents','count','confidence','support','lift','conviction']]
    return basket

### 1. 카테고리 가져오기

In [4]:
#카테고리 가져오기
rex= pd.read_clipboard(sep='\t')
rex=rex.astype(str)
rex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   depth1            41 non-null     object
 1   Category1D        41 non-null     object
 2   Category2D        41 non-null     object
 3   Category2D_Value  41 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


### 2. 한총 데이터 (월별 데이터)

In [5]:
df= pd.read_clipboard(sep='\t')
df=df[1:]
df=df.reset_index().rename(columns={"index": "Model Code (v41)"})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6507 entries, 0 to 6506
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Model Code (v41)        6507 non-null   object
 1   Order (purchase event)  6507 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 101.8+ KB


In [10]:
df

,Model Code (v41),Order (purchase event)
0,RF85B90023Y,1242
1,VS20B957D5G,980
2,VCA-ADB95A,685
3,HAF-HIN,667
4,VCA-SBT90/VT,658
...,...,...
5486,"RF85B90023Y,RQ48B94M1AP01",1
5487,"AP145BSPPHH7SY,AP110BSPPHH8SY",1
5488,"SM-R870NZSAKOO,SM-R180NZNAKOO",1
5489,"AX053B810HND,HKSOUNDSTICK4WHTAS",1


In [6]:
# 동시구매 
#month_df=df[df['Model Code (v41)'].str.contains(',')]
#month_df=month_df[month_df[month_df.columns[1]]!=0]  
#month_df=month_df.reset_index(drop=True)
month_df=df
#전체 SKU 하나의 리스트로
a = month_df['Model Code (v41)'].tolist()
b = ','.join(a) #리스트 문자열로
c = b.split(',') #문자열 개별 문자로 리스트화 
model = list(set(c)) #중복 제거

### 3. 동시구매 :부문별 통계

In [9]:
d_mod=[]
for cate_type1, cate_type2 in zip(["MX","CE","MX"],["MX","CE","CE"]):
    print(f'<<<{cate_type1}+{cate_type2}>>')
    new_rex=rex[(rex['depth1']==cate_type1)|(rex['depth1']==cate_type2)]
    new_rex=new_rex.reset_index()

    ## 1. 카테고리 생성 : 부문_1depth 카테고리_2depth 카테고리
    dic_category = {}
    for m in model :
        flag = 1
        for i in range(len(new_rex)):
            r = re.compile(new_rex.Category2D_Value[i])
            if r.search(m):
                dic_category[m] = new_rex.depth1[i]+'_'+ new_rex.Category1D[i] +'_'+ new_rex.Category2D[i]
                flag = 0
                break
        if flag :
            dic_category[m] = '기타'
    
    ## 2. 데이터셋 만들기 : 부문별
    dataset_type=dataset_make(dic_category, month_df)
    print('동시구매 order 수', len(dataset_type))
    print('동시구매 비율', (len(dataset_type)/df[df.columns[1]].sum()*100))

    ## 3. 부문별 함께 구매
    select_1=[s for s in dataset_type if any(cate_type1 in l for l in s)]
    result=[s for s in select_1 if any(cate_type2 in l for l in s)]

    ## 4. 기타 제거 : 해당하는 부문 분석 외에는 기타로 나오기 때문에 기타만 제거
    #1) 행제거
    if cate_type1==cate_type2:
        print('MX 또는 CE 분석으로 진행')
        ho = []
        rm_set = {'기타'} 
        for s in result:
            if all(p not in rm_set for p in s):
                ho.append(s)

    #2) 시각적 제거
    if cate_type1!=cate_type2:
        print('MX+CE 분석으로 진행')
        ho = []
        rm_set = {'기타'} 
        for s in result:
            arr_new = [i for i in s if i not in rm_set]
            ho.append(arr_new)

    dataset_type = ho
    print(f'{cate_type1}+{cate_type2}_기타 제거후 주문건수: ',len(dataset_type))
    d_mod.append(len(dataset_type))
    print('-------------------')

    ## 5-1. 분석 :부문별 1depth 주문건수 
    order_1depth(new_rex, dataset_type).sort_values(["count"],ascending=False).to_csv(f'{path}/{file_name}/주문 건수_1depth_{cate_type1}+{cate_type2}.csv',index=False, encoding='utf-8-sig')

    ## 5-2. 분석 : 부문별 2depth 제품건수
    product_2depth(new_rex, dataset_type).to_csv(f'{path}/{file_name}/제품 건수_2depth_{cate_type1}+{cate_type2}.csv',index=False, encoding='utf-8-sig')

    ######################### 참고용  ########################
    # 6. 조합 결과 
    # count=Counter([tuple(sorted(i)) for i in sorted([tuple(set(item)) for item in  dataset_type])]) 
    # da=sorted(count.items(), key=lambda i: i[1], reverse=True)
    # #조합별 카운트 데이터 저장
    # pd.DataFrame(da).to_csv(f'{path}/{file_name}/조합결과_{cate_type1}+{cate_type2}.csv', encoding='utf-8-sig')

    # 7. 장바구니 분석
    #my_basket(dataset_type).to_csv(f'{path}/{file_name}/장바구니 분석_{cate_type1}+{cate_type2}.csv', encoding = 'utf-8-sig')


<<<MX+MX>>
동시구매 order 수 6236
동시구매 비율 10.895240757565169
MX 또는 CE 분석으로 진행
MX+MX_기타 제거후 주문건수:  1602
-------------------
<<<CE+CE>>
동시구매 order 수 6236
동시구매 비율 10.895240757565169
MX 또는 CE 분석으로 진행
CE+CE_기타 제거후 주문건수:  3514
-------------------
<<<MX+CE>>
동시구매 order 수 6236
동시구매 비율 10.895240757565169
MX+CE 분석으로 진행
MX+CE_기타 제거후 주문건수:  103
-------------------


In [10]:
###### 부문별 통계량 ###### 
d_mod_new_=[]
for y  in df.columns[1:]:
    all_or=df[y].sum() #전체
    im_order=d_mod[0] 
    ce_order=d_mod[1]
    ce_im_order=d_mod[2]
    two_sum=month_df[y].sum()  #공동구매
    ratio_two=two_sum/df[y].sum()  #공동구매 비율
    etc= two_sum -(ce_im_order + ce_order + im_order) #기타
    d_mod_new_1=[]
    for i in [ce_im_order,  ce_order, im_order, ratio_two, all_or, two_sum, etc]:
        d_mod_new_1.append(i)
    d_mod_new_.append(d_mod_new_1)

    
pd.DataFrame(d_mod_new_,columns=['CE+MX','CE+CE','MX+MX','공동구매 비율',"전체 주문수",'공동구매(2개이상)','기타'],index=[num for num in df.columns[1:]]).to_excel(f'{path}/{file_name}/[주문수]부문별 통계.xlsx')
pd.DataFrame(d_mod_new_,columns=['CE+MX','CE+CE','MX+MX','공동구매 비율',"전체 주문수",'공동구매(2개이상)','기타'])


,CE+MX,CE+CE,MX+MX,공동구매 비율,전체 주문수,공동구매(2개이상),기타
0,103,3514,1602,0.108952,57236,6236,1017


### 4-1. 2depth 장바구니 분석

In [7]:
## 1. 카테고리 생성 : 부문별_카테고리 1depth_카테고리 2depth
dic_category_2depth = {}
for m in model :
    flag = 1
    for i in range(len(rex)):
        r = re.compile(rex.Category2D_Value[i])
        if r.search(m):
            dic_category_2depth[m] = rex.depth1[i]+'_'+rex.Category1D[i] +'_'+ rex.Category2D[i]  
            flag = 0
            break
    if flag :
        dic_category_2depth[m] = '기타'

## 2. 데이터셋
dataset_2depth=dataset_make(dic_category_2depth, month_df)

## 3. 기타: 시각적 제거 -> 모든 카테고리를 다 불러오므로
# ho = []
# rm_set = {'nan','기타'}
# for s in dataset_2depth:
#     arr_new = [i for i in s if i.split('_')[0] not in rm_set]
#     ho.append(arr_new)
# dataset_2depth=ho

## 4-1. 분석 : 1depth 주문건수
order_1depth(rex, dataset_2depth).sort_values(["count"]).to_csv(f'{path}/{file_name}/주문 건수_2depth_전체.csv',index=False, encoding='utf-8-sig')

## 4-2. 분석 : 2depth 제품건수
product_2depth(rex, dataset_2depth).to_csv(f'{path}/{file_name}/제품건수_2depth_전체.csv',index=False, encoding='utf-8-sig')

## 4-3. 분석 : 장바구니 분석
my_basket(dataset_2depth).to_excel(f'{path}/{file_name}/[월간보고]동시구매2depth_장바구니 분석.xlsx',index=False)

# 5. 조합 결과 : 유니크
count_unique=Counter([tuple(sorted(i)) for i in sorted([tuple(set(item)) for item in  dataset_2depth])]) 
da_unique=sorted(count_unique.items(), key=lambda i: i[1], reverse=True)

pd.DataFrame(da_unique).to_csv(f'{path}/{file_name}/조합결과_unique.csv', encoding='utf-8-sig')


### 4-2 액세서리 시각적 제거후, 2개이상의 주문리스트만 장바구니 분석

In [12]:

## 1. 기타: 시각적 제거
only_product = []
pro_type = {'액세서리'}
for s in dataset_2depth:
    prd_new = [i for i in s if i[-4:] not in pro_type]
    only_product.append(prd_new)

final_only_df=list(filter(lambda x: len(x)>=2, only_product)) 

## 2. 분석 : 장바구니 분석
my_basket(final_only_df).to_excel(f'{path}/{file_name}/[액세서리제거]2depth_장바구니 분석.xlsx',index=False)

In [13]:
len(final_only_df)

2201

### 5. 1depth 조합 장바구니 분석

In [66]:
## 1. 카테고리 생성 : 카테고리 1depth
dic_category_1depth = {}
for m in model :
    flag = 1
    for i in range(len(rex)):
        r = re.compile(rex.Category2D_Value[i])
        if r.search(m):
            dic_category_1depth[m] = rex.Category1D[i]
            flag = 0
            break
    if flag :
        dic_category_1depth[m] = '기타'

## 2. 데이터셋 
dataset_1depth=dataset_make(dic_category_1depth, month_df)

## 3.기타: 시각적 제외
ho = []
rm_set = {'nan','기타'}
for s in dataset_1depth:
    arr_new = [i for i in s if i.split('_')[0] not in rm_set]
    ho.append(arr_new)

dataset_1depth=ho

## 4. 분석 : 장바구니 분석
my_basket(dataset_1depth).to_excel(f'{path}/{file_name}/[월간보고]1depth_장바구니 분석.xlsx',index=False)

### 참고용
### <장바구니 분석 및 통계분석 기준>
- 2023.01.05

1) MX + MX  ( MX 단독 구매): only MX으로만 구성
  - 기타 제거: 기타 "행" 제거      
    - 예시) [MX, MX, 기타 ] -> 행 제거됨

2) CE+CE (CE 단독 구매)
  - 1번과 방식 동일

3) MX+CE / TV+생활가전 등 : 각 1개 이상 필수 포함
  - 기타 제거: **시각적** 기타 제거
    - 예시) [TV, 세탁기, 기타]   -> [TV, 세탁기]
    - 예시) [TV, 세탁기, 냉장고 ] -> [TV, 세탁기, 냉장고 ]와 같은 형태 포함됨

4) 전체 (2depth)
  - 기타 제거 :**시각적** 기타제거